# Compare before & after CB

In [1]:
import os, sys, re, io, math
import matplotlib, shap, xgboost
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import *
from math import isnan, nan
from matplotlib.widgets import Slider
from glob import glob
from ipywidgets import *
from datetime import datetime, timedelta
from dateutil.tz import tzlocal
import ipywidgets as widgets
from IPython.display import *
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.options.display.width = 1000
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 100000
pd.options.display.float_format = '{0:f}'.format

# Disable scrolling for all output widgets
style = """
    <style>
       .jupyter-widgets-output-area .output_scroll {
            height: unset !important;
            border-radius: unset !important;
            -webkit-box-shadow: unset !important;
            box-shadow: unset !important;
        }
        .jupyter-widgets-output-area  {
            height: auto !important;
        }
    </style>
    """
display(HTML(style))

# disable autosave so that multiple user browser can attach the same notebook
%autosave 0

from pandas_serializer import *
from compare_CB import *

CB_start_date = pd.Timestamp('2020-04-07', tz='tzlocal()')
CB_boundary_gap = pd.to_timedelta('3D')  # at least N days from CB start date
CB_boundary_end = pd.to_timedelta('45D')  # at most N days from CB start date

DateRangeA = [CB_start_date-CB_boundary_end, CB_start_date-CB_boundary_gap]
DateRangeB = [CB_start_date+CB_boundary_gap, CB_start_date+CB_boundary_end]

Autosave disabled


In [2]:
button = widgets.Button(description='Load Data', style={'font_weight': 'bold'})
def on_button_clicked(b):
    global all_data
    for f in ['5.decrypted/izedAa85XXrDS85XlwrOsIDU/all-data.pson.gz', '5.decrypted/q8KKsBwu0cryrVM3VMBNW35Q/all-data2.pson.gz']:
        if os.path.exists(f):
            print('Loading files ...', end='')
            all_data = pandas_load(f)
            print('Done')
            break
button.on_click(on_button_clicked)
display(button)

Button(description='Load Data', style=ButtonStyle(font_weight='bold'))

In [3]:
button1 = widgets.Button(description='Set Comparison Date-range', style={'font_weight': 'bold'}, layout=Layout(width='250px'))
out1 = widgets.Output()
datepicker4 = [widgets.DatePicker(value=val, description=desc, layout=Layout(width='250px'), disabled=False) for desc,val in 
               [['RangeA Start', DateRangeA[0]], ['RangeA Stop', DateRangeA[1]], ['RangeB Start', DateRangeB[0]], ['RangeB Stop', DateRangeB[1]]]]
@out1.capture(clear_output=True)
def button_set_compare_range(b):
    global DateRangeA, DateRangeB, out
    dates = [pd.to_datetime(i.value) for i in datepicker4]
    if dates[0:2].count(None)>1 or dates[2:4].count(None)>1:
        return
    DateRangeA, DateRangeB = dates[0:2], dates[2:4]
    print('DateRange set to A=[%.10s, %.10s), B=[%.10s, %.10s)'%(DateRangeA[0], DateRangeA[1], DateRangeB[0], DateRangeB[1]))
button1.on_click(button_set_compare_range)
display(HBox(datepicker4+[button1]), out1)

Output()

# Compare stats

In [4]:
@interact(show=widgets.ToggleButton(value=False,description='Show Comparison'))
def show_compare(show):
    if not show:
        return clear_output()
    display(get_compare(all_data, CB_start_date, CB_boundary_gap, CB_boundary_end))

interactive(children=(ToggleButton(value=False, description='Show Comparison'), Output()), _dom_classes=('widg…

# Plot Shap

In [5]:
@interact(figwidth=widgets.IntText(value=20, description='Figure width'), show=widgets.ToggleButton(value=False,description='Show Shap Plot'))
def show_shap(figwidth, show):
    if not show:
        return clear_output()
    f, shap_values, X = get_shap(all_data, CB_start_date, CB_boundary_gap, CB_boundary_end, figwidth=figwidth, max_display=9999)
    # f.savefig("/summary_plot1.png", bbox_inches='tight', dpi=600)

interactive(children=(IntText(value=20, description='Figure width'), ToggleButton(value=False, description='Sh…

# For Testing

In [6]:
for ii,(name, df) in enumerate(all_data.items()):
    if ii!=2 or True:
        continue
    df1 = df[[col for col in df.columns if (col.startswith('tapsLog') and 'dur' in col)]]
    display(name)
    display(df1.max())


NameError: name 'all_data' is not defined